In [1]:
import gc
import os
from datetime import datetime
import pickle
import psutil
from tqdm import tqdm, tqdm_notebook
from functools import partial, update_wrapper
from itertools import product
from multiprocessing import Pool

import bloscpack as bp

import random
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

from YSMLT import utils as g_utils
from YSMLT.series import utils as ts_utils

from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy.signal import welch, find_peaks
from scipy import stats
from scipy.special import entr
from scipy.stats import entropy
from scipy.stats import percentileofscore
from tsfresh.feature_extraction import feature_calculators

### mean -0.31079195780498403
### std  2.4983141023158186

In [6]:
np.nanmax(pdf_trn_trnsfrmr)

inf

In [4]:
pdf_trn = pd.read_pickle('../input/train_clean_encoded.pkl')
pdf_tst = pd.read_pickle('../input/test_clean_encoded.pkl')
pdf_trn_trnsfrmr = bp.unpack_ndarray_from_file(os.path.join('../input', 'trn_dat_kal_all.bp'))
pdf_tst_trnsfrmr = bp.unpack_ndarray_from_file(os.path.join('../input', 'tst_dat_kal_all.bp'))

# for i in range(3):
#     avg = pdf_trn_trnsfrmr[:, i].mean()
#     std = pdf_trn_trnsfrmr[:, i].std()
#     pdf_trn_trnsfrmr[:, i] = (pdf_trn_trnsfrmr[:, i] - avg) / std
#     pdf_tst_trnsfrmr[:, i] = (pdf_tst_trnsfrmr[:, i] - avg) / std

# with open('../input/batch_ids_trn.pkl', 'rb') as f:
#     batch_id_trn = pickle.load(f)
# with open('../input/batch_ids_tst.pkl', 'rb') as f:
#     batch_id_tst = pickle.load(f)
    
# with open('../input/target_codes.pkl', 'rb') as f:
#     encoder = pickle.load(f)

# pdf_trn['signal_49'] = pdf_trn_trnsfrmr[:, 0]
# pdf_trn['signal_50'] = pdf_trn_trnsfrmr[:, 1]
# pdf_trn['signal_51'] = pdf_trn_trnsfrmr[:, 2]

# pdf_tst['signal_49'] = pdf_tst_trnsfrmr[:, 0]
# pdf_tst['signal_50'] = pdf_tst_trnsfrmr[:, 1]
# pdf_tst['signal_51'] = pdf_tst_trnsfrmr[:, 2]

In [3]:
pdf_trn

,time,signal,open_channels,signal_0,signal_1,signal_2,signal_3,signal_4,signal_5,signal_6,...,signal_42,signal_43,signal_44,signal_45,signal_46,signal_47,signal_48,signal_49,signal_50,signal_51
0,0.0001,-2.760000,0,0.435937,0.0,0.453637,0.000000,0.462303,0.000000,0.470284,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,-2.780745,-0.020745,0.007516
1,0.0002,-2.855700,0,0.292407,0.0,0.308404,0.000000,0.315577,0.000000,0.335668,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,-2.784894,0.070806,-0.024795
2,0.0003,-2.407400,0,0.897187,0.0,0.902944,0.000259,0.908428,0.000139,0.885295,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,-2.781962,-0.374562,0.155588
3,0.0004,-3.140400,0,0.045574,0.0,0.050236,0.000000,0.051367,0.000000,0.066310,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,-2.816487,0.323913,-0.103144
4,0.0005,-3.152500,0,0.041195,0.0,0.045377,0.000000,0.046395,0.000000,0.061386,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,-2.818621,0.333879,-0.105909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,499.9996,2.919274,7,0.999996,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.0,1.0,0.990843,0.267996,0.000131,0.000000,0.000000,3.921013,1.001739,0.343147
4999996,499.9997,2.697906,7,0.999996,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.0,1.0,0.966391,0.124179,0.000008,0.000000,0.000000,4.018794,1.320889,0.489598
4999997,499.9998,4.516337,8,0.999998,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.0,1.0,1.000000,0.999254,0.604478,0.003535,0.000000,4.248664,-0.267673,-0.059268
4999998,499.9999,5.639669,9,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.0,1.0,1.000000,1.000000,0.998614,0.500105,0.001428,4.451766,-1.187902,-0.210633


In [16]:
step = 500
wndw = 500
tag = 'last_transformer'

In [17]:
all_trn_dat, all_trn_lbl = make_srs_feats(pdf_trn, batch_id_trn, step=step, wndw=wndw)

100%|██████████| 10/10 [00:26<00:00,  2.66s/it]


In [18]:
bp.pack_ndarray_to_file(all_trn_dat, '../input/feats_srs/trn_srs_dat_all_s{:d}_w{:d}_feat_{}.bp'.format(step, wndw, tag))
bp.pack_ndarray_to_file(all_trn_lbl, '../input/feats_srs/trn_srs_lbl_all_s{:d}_w{:d}_feat_{}.bp'.format(step, wndw, tag))

In [19]:
all_tst_dat = make_srs_feats(pdf_tst, batch_id_tst, step=step, wndw=wndw, mode='test')

100%|██████████| 20/20 [00:07<00:00,  2.53it/s]


In [20]:
bp.pack_ndarray_to_file(all_tst_dat, '../input/feats_srs/tst_srs_dat_all_s{:d}_w{:d}_feat_{}.bp'.format(step, wndw, tag))

In [10]:
def make_srs_feats(raw_pdf, batch_ids, step, wndw, mode='train'):
    
    outputs_dat = []
    if mode == 'train':
        outputs_lbl = []
    
    for i in tqdm(range(0, len(batch_ids))):
        sgnl_ndcs = batch_ids[i]

        # ------------------------------------------------
        stack_collection = []
        sgnl = raw_pdf['signal'].iloc[sgnl_ndcs]
        stack_collection.append(
            ts_utils.serie_stacker(sgnl, step_size=step, block_size=wndw)
        )
        
        for j in range(52):
            sgnl = raw_pdf['signal_' + str(j)].iloc[sgnl_ndcs]
            stack_collection.append(
                ts_utils.serie_stacker(sgnl, step_size=step, block_size=wndw)
            )

        outputs_dat.append(np.stack(stack_collection, axis=2))
        
        if mode == 'train':
            trgt = raw_pdf['open_channels'].iloc[sgnl_ndcs]
            outputs_lbl.append(
                ts_utils.serie_stacker(trgt, step_size=step, block_size=wndw)
            )
            
    if mode == 'train':
        return np.concatenate(outputs_dat, 0), np.concatenate(outputs_lbl, 0)
    else:
        return np.concatenate(outputs_dat, 0)